求脑袋之间的距离（1-corr）之后求相关

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import scipy.io as sio

main_folder = '24.12.17-IS-RSA'
outdatapath = '24.12.17-IS-RSA/'
beha_data = sio.loadmat(os.path.join(outdatapath, 'beha_data_py.mat'))
beha_data = beha_data.get('beha_data', None)
print(beha_data)
print(beha_data.shape)
sub_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]

In [ ]:
def calculate_correlation(y):
    R = 1 - np.corrcoef(y)
    R_1 = np.tril(R)
    R_1[np.triu_indices_from(R_1, 0)] = 99999
    R_1 = R_1.T
    return R_1[R_1 < 2]

for sub_folder in sub_folders:
    folder_path = os.path.join(main_folder, sub_folder)
    files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    
    all_brain_data = []
    brain_values_name = []

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        raw_data = pd.read_table(file_path, header=None, sep=r'\s+')
        raw_data = raw_data.iloc[3:, :]
        if raw_data.shape[1] <= 30:
            print(f"Skipping {file_name} as it has {raw_data.shape[1]} columns.")
            continue
        R_2 = calculate_correlation(raw_data.values)

        brain_values_name.append([os.path.splitext(file_name)[0]] + R_2.tolist())
        all_brain_data.extend(R_2)
        print(all_brain_data)
        print(sub_folder,file_name)

    sio.savemat(os.path.join(outdatapath, f'{sub_folder}_brain_list.mat'), {'brain_list': files})
    sio.savemat(os.path.join(outdatapath, f'{sub_folder}_all_brain_data.mat'), {'all_brain_data': all_brain_data})

    if beha_data is not None:
        r_values, p_values = pearsonr(beha_data.flatten(), all_brain_data)
        rsps = np.array([files, r_values, p_values]).T
        sio.savemat(os.path.join(outdatapath, f'{sub_folder}_rsps.mat'), {'rsps': rsps})
    else:
        print(f"Warning: 'beha_data' not found in GPT.mat for {sub_folder}, skipping RSA calculation.")